<a href="https://colab.research.google.com/github/alondraortegaherrera/coloreado_grafos/blob/main/coloreado_de_grafos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\color{DarkSlateGray}{\huge\text{Coloreado de Grafos con Recocido Simulado}}$

In [195]:
# @title **Librerías**
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import shuffle, sample
import math

In [196]:
# @title **Problema**

"""
    El problema consiste en encontrar una asignación de colores para los vértices de un grafo de acuerdo con una
    secuencia de grados dada, con el objetivo de minimizar los conflictos entre los vértices adyacentes.
    Cada vértice debe ser asignado un color de un conjunto limitado de colores (en este caso, hasta 4 colores),
    de forma que los vértices adyacentes no compartan el mismo color.
"""

class ColoreadoGrafo():
    def __init__(self, degree_seq, num_min_colores=4):
        self.num_min_colores = num_min_colores
        self.grafo = self.generar_grafo(degree_seq)
        self.grafo_dict = self.diccionario_grafo(self.grafo)
        self.num_vertices = len(self.grafo.nodes())

    def generar_grafo(self, degree_seq):
        # Suma de grados debe ser par - se debe cumplir "Handshaking Lemma"
        if sum(degree_seq) % 2 != 0:
            raise ValueError("Suma de grados no válida, la suma de todos los grados debe ser par...")

        G = nx.configuration_model(degree_seq)
        # Hacerlo un grafo simple - eliminar loops y aristas paralelas
        G = nx.Graph(G)
        G.remove_edges_from(nx.selfloop_edges(G))

        return G

    def diccionario_grafo(self, G):
        grafo_dict = {}

        colores_disponibles = list(range(1, self.num_min_colores + 1))  # Del 1 al 4
        shuffle(colores_disponibles)  # Mezclar colores

        for vertice in G.nodes():
            adyacentes = set(G.neighbors(vertice)) # Conjunto de vértices adyacentes
            if len(colores_disponibles) > 0:       # Asegurar que se usen los 4 colores
                color = colores_disponibles.pop()
            else:
                color = random.randint(1, self.num_min_colores)  # Asignar aleatorios (1 al 4) para los que faltan
            grafo_dict[vertice] = (adyacentes, color, 0)     # Inicializar conflictos en 0

        self.contar_conflictos(grafo_dict)

        return grafo_dict

    def contar_conflictos(self, grafo_dict):
      for vertice in grafo_dict:
          vecinos = grafo_dict[vertice][0]
          color = grafo_dict[vertice][1]

          # Resetear conflictos antes de recalcular
          conflictos = 0
          for vecino in vecinos:
              color_vecino = grafo_dict[vecino][1]

              if color_vecino == color:
                  conflictos += 1

          grafo_dict[vertice] = (vecinos, color, conflictos)

    def calcular_conflictos_totales(self):
      suma_conflictos = 0

      for vertice in self.grafo_dict:
          conflictos = self.grafo_dict[vertice][2]
          suma_conflictos += conflictos  # Acumular los conflictos

      return suma_conflictos

    def imprimir_dict_grafo(self):
        print("Diccionario del grafo:")
        for i, (k, v) in enumerate(self.grafo_dict.items()):
            print(f"{k}: {v}", end="  ")
            if (i + 1) % 1 == 0:
                print()
        print()

    def plot_grafo(self):
        plt.figure(figsize=(6,6))
        pos = nx.spring_layout(self.grafo)
        color_vertice = []

        for vertice in self.grafo.nodes():
            color = self.grafo_dict[vertice][1]
            color_vertice.append(color)

        nx.draw(self.grafo, pos, with_labels=True, node_color=color_vertice, cmap=plt.cm.viridis, edge_color='gray', node_size=700, font_size=12, font_color='white')
        plt.title("Coloreado del Grafo")
        plt.show()

In [197]:
# @title **Secuencias válidas para generar el grafo**

def generar_secuencia_grados(num_vertices, min_grado, max_grado):
    grados = []

    for _ in range(num_vertices):
        grado_aleatorio = random.randint(min_grado, max_grado)
        grados.append(grado_aleatorio)

    suma_grados = sum(grados)
    if suma_grados % 2 != 0:  # Hacer par si es impar
        grados[-1] -= 1

    return grados

#degree_seq = generar_secuencia_grados(10, 2, 6)
#print("Usar secuencia:", degree_seq)

In [198]:
# @title **Cambio de temperatura - Lineal Exponencial**
def cambio_de_temperatura(temperatura_actual,opcion='lineal',factor_enfriamiento=0.98):
    if opcion == 'lineal':
        return temperatura_actual*(1-(0.951-factor_enfriamiento))
    elif opcion == 'exponencial':
        return temperatura_actual * factor_enfriamiento

def get_curva(temperatura_actual=1000,opcion='lineal',factor_enfriamiento=0.98,delta=1):
    nueva_temp=[]
    prob_ac = []
    temp_act = temperatura_actual
    for i in range(100):
      temp_act = cambio_de_temperatura(temp_act,opcion,factor_enfriamiento)
      nueva_temp.append(temp_act)
      prob_ac.append(math.exp(-delta / temp_act))
    return nueva_temp,prob_ac

temp, proba = get_curva(factor_enfriamiento=0.98)
print(temp[-1])

temp, proba = get_curva(opcion='exponencial',factor_enfriamiento=0.98)
print(temp[-1])

17439.638415178048
132.61955589475298


In [199]:
# @title **Recocido Simulado**
class SimulatedAnealingCG(ColoreadoGrafo):
    def __init__(self, degree_seq, num_min_colores = 4, max_iter = 1000, temp_ini =5000):
        super().__init__(degree_seq, num_min_colores)
        self.max_iteraciones = max_iter
        self.temperatura_inicial = temp_ini
        self.opcion = 'exponencial'
        self.factor_enfriamiento = 0.98

    def set_parametros(self, temp = 1000, opcion = 'exponencial', fe = 0.98, max_it = 1000):
        self.temperatura_inicial = temp
        self.opcion = opcion
        self.factor_enfriamiento = fe
        self.max_iteraciones = max_it

    def cambio_de_temperatura(self, temperatura_actual):
        if self.opcion == 'lineal':
            return temperatura_actual * (1 - (0.951 - self.factor_enfriamiento))
        elif self.opcion == 'exponencial':
            return temperatura_actual * self.factor_enfriamiento

    def crear_vecino(self):
      nodos_con_conflicto = []

      for vertice in self.grafo_dict:
          if self.grafo_dict[vertice][2] > 0:
              nodos_con_conflicto.append(vertice)

      if len(nodos_con_conflicto) > 0:
          vertice = random.choice(nodos_con_conflicto)
      else:
          vertice = random.choice(list(self.grafo_dict.keys()))

      vecinos = self.grafo_dict[vertice][0]
      color_actual = self.grafo_dict[vertice][1]

      colores_vecinos = set()  # no repetir colores

      for vecino in vecinos:
          color_vecino = self.grafo_dict[vecino][1]
          colores_vecinos.add(color_vecino)

      colores_disponibles = []

      for color in range(1, self.num_min_colores + 1):
          if color != color_actual and color not in colores_vecinos:
              colores_disponibles.append(color)

      # cambiar el color del nodo si hay colores disponibles
      if len(colores_disponibles) > 0:
          nuevo_color = random.choice(colores_disponibles)
          self.grafo_dict[vertice] = (vecinos, nuevo_color, 0)

          self.contar_conflictos(self.grafo_dict)  # Volver a contar conflictos

    def recocido_simulado(self):
        temperatura_actual = self.temperatura_inicial
        iteraciones = 0
        costo_actual = self.calcular_conflictos_totales()  # del grafo inicial

        while temperatura_actual > 0.1 and iteraciones < self.max_iteraciones:
            self.crear_vecino()
            costo_vecino = self.calcular_conflictos_totales()

            delta_costo = costo_vecino - costo_actual
            if delta_costo < 0 or random.random() < math.exp(-delta_costo / temperatura_actual):
                costo_actual = costo_vecino

            temperatura_actual = self.cambio_de_temperatura(temperatura_actual)
            iteraciones += 1
            print(f"Iteración {iteraciones}: Costo actual: {costo_actual}, Temperatura: {temperatura_actual:.2f}")

        return self.grafo_dict, costo_actual

degree_seq = generar_secuencia_grados(500, 3, 4)
grafo_simulado = SimulatedAnealingCG(degree_seq, num_min_colores=4)
grafo_coloreado, costo_final = grafo_simulado.recocido_simulado()

Iteración 1: Costo actual: 442, Temperatura: 4900.00
Iteración 2: Costo actual: 440, Temperatura: 4802.00
Iteración 3: Costo actual: 436, Temperatura: 4705.96
Iteración 4: Costo actual: 434, Temperatura: 4611.84
Iteración 5: Costo actual: 430, Temperatura: 4519.60
Iteración 6: Costo actual: 428, Temperatura: 4429.21
Iteración 7: Costo actual: 426, Temperatura: 4340.63
Iteración 8: Costo actual: 424, Temperatura: 4253.82
Iteración 9: Costo actual: 420, Temperatura: 4168.74
Iteración 10: Costo actual: 418, Temperatura: 4085.36
Iteración 11: Costo actual: 416, Temperatura: 4003.66
Iteración 12: Costo actual: 414, Temperatura: 3923.58
Iteración 13: Costo actual: 410, Temperatura: 3845.11
Iteración 14: Costo actual: 406, Temperatura: 3768.21
Iteración 15: Costo actual: 404, Temperatura: 3692.85
Iteración 16: Costo actual: 400, Temperatura: 3618.99
Iteración 17: Costo actual: 396, Temperatura: 3546.61
Iteración 18: Costo actual: 390, Temperatura: 3475.68
Iteración 19: Costo actual: 388, Temp

In [200]:
# @title **Resultado**
grafo_simulado.imprimir_dict_grafo()
print("Costo final:", costo_final)
#grafo_simulado.plot_grafo()

Diccionario del grafo:
0: ({289, 195, 109}, 4, 0)  
1: ({296, 249, 422, 263}, 2, 0)  
2: ({359, 351, 215}, 3, 0)  
3: ({384, 489, 99, 372}, 2, 0)  
4: ({432, 130, 413}, 3, 0)  
5: ({313, 90, 379, 494}, 1, 0)  
6: ({473, 164, 326, 231}, 3, 0)  
7: ({321, 274, 20, 334}, 3, 0)  
8: ({392, 383, 303}, 3, 0)  
9: ({344, 321, 418, 161}, 3, 0)  
10: ({32, 425, 368, 15}, 3, 0)  
11: ({202, 171, 404, 270}, 2, 0)  
12: ({457, 235, 52, 398}, 4, 0)  
13: ({72, 241, 267, 117}, 3, 0)  
14: ({269, 419, 325}, 4, 0)  
15: ({104, 10, 372}, 1, 0)  
16: ({224, 225, 147}, 2, 0)  
17: ({400, 161, 164, 349}, 4, 0)  
18: ({410, 243, 229, 151}, 2, 0)  
19: ({394, 262, 254, 151}, 4, 0)  
20: ({249, 203, 342, 7}, 4, 0)  
21: ({73, 212, 389, 54}, 2, 0)  
22: ({144, 49, 88, 383}, 2, 0)  
23: ({448, 458, 94, 402}, 1, 0)  
24: ({432, 169, 95}, 1, 0)  
25: ({146, 324, 228, 206}, 4, 0)  
26: ({192, 88, 147, 335}, 1, 0)  
27: ({434, 333, 422}, 1, 0)  
28: ({346, 100, 276}, 2, 0)  
29: ({392, 379, 51, 350}, 1, 0)  
30: (